<u>Imports</u>

In [1]:
#!pip install openai==1.36.0

In [2]:
import openai
import re
from collections import Counter

<u>Parameters</u>

In [3]:
# Initialize model parameters 

openai.api_key = open("key.txt","r").read().strip("\n")
model = "gpt-4-turbo"
temperature = 0
max_tokens = 4095

In [4]:
# Initalize system prompt

system_msg = {"role": "system", "content": """
Tu es un journaliste. 
Ton objectif est de comprendre un texte, le synthétiser et générer des questions.
Les textes transmis seront délimités par ces caractères : <<< (début), >>> (fin).
Utilise un langage clair et élégant sans commentaire ou mise en forme.
"""}

In [5]:
# Import test text

text = open("camus-tipasa.txt","r").read()

<u>Mission 1: extract key topics</u>

In [6]:
nb_themes = 5

In [7]:
user_msg_themes = {"role": "user", "content":f"""
Tu reçois le texte suivant : <<< {text} >>>
Propose une liste de {nb_themes} thèmes basés uniquement sur ce thème. Chaque thème doit couvrir un aspect distinct du texte. 
Chaque thème sera séparé par deux retours à la ligne (\n\n). 
"""}

In [8]:
completion1 = openai.chat.completions.create(
    model=model,
    messages=[system_msg,user_msg_themes],
    temperature=temperature,
    max_tokens=max_tokens)

In [9]:
answer1 = completion1.choices[0].message.content
print(answer1)
topics = answer1.split('\n\n')

1. La fusion de l'homme avec la nature : Le texte explore profondément la connexion entre l'homme et la nature, illustrant comment le narrateur s'intègre et s'harmonise avec le paysage, les plantes, et les éléments naturels de Tipasa.

2. La spiritualité et la présence divine dans le quotidien : Le récit souligne la présence des dieux dans les aspects quotidiens de la vie et du paysage, suggérant une dimension sacrée et spirituelle imbriquée dans la nature et les ruines de Tipasa.

3. La réminiscence et la transformation des ruines : Les ruines, centrales dans le texte, symbolisent à la fois le passé historique et culturel et leur renaissance au sein de la nature, marquant un cycle de destruction et de renouveau.

4. La sensorialité et l'expérience immersive : Le texte est riche en descriptions sensorielles qui invitent le lecteur à expérimenter Tipasa à travers les odeurs, les couleurs, les textures et les sons, accentuant l'expérience immersive du narrateur.

5. La contemplation et l

<u>Mission 2: interview guide on each theme</u>

In [10]:
nb_questions = 5
    
user_msg_questions = {"role": "user", "content": f"""
Propose une liste de {nb_questions} questions sur chaque thème mentionné.
Ces questions doivent être factuelles et descriptives. 
"""}

completion2 = openai.chat.completions.create(
    model=model,
    messages=[system_msg,user_msg_themes,{"role":"assistant","content":answer1},user_msg_questions],
    temperature=temperature,
    max_tokens=max_tokens)

answer2 = completion2.choices[0].message.content
print(answer2)

1. La fusion de l'homme avec la nature :
- Quels éléments naturels le narrateur décrit-il en détail dans le texte ?
- Comment le narrateur décrit-il son interaction physique avec les plantes de Tipasa ?
- De quelle manière les sensations physiques du narrateur sont-elles utilisées pour illustrer sa connexion avec la nature ?
- Quels sont les effets de la nature sur l'état émotionnel du narrateur ?
- Comment le narrateur utilise-t-il les éléments naturels pour décrire ses sentiments et expériences ?

2. La spiritualité et la présence divine dans le quotidien :
- Quels dieux sont mentionnés ou suggérés dans le texte ?
- Comment la présence des dieux est-elle intégrée dans les descriptions du paysage de Tipasa ?
- De quelle manière le narrateur perçoit-il la spiritualité dans son environnement ?
- Quels aspects de la vie quotidienne à Tipasa sont décrits comme étant influencés par le divin ?
- Comment la nature et les ruines contribuent-elles à l'ambiance spirituelle du lieu ?

3. La rémi

<u>Mission 3: questionnaire and voting</u>

In [11]:
# Set parameters

nb_repetitions = 5
temperature = 0.5

In [12]:
# Set prompt, asking openai to boxed answers (to enable easy extraction)

instructions = f"Tu reçois ce texte : <<< {text} >>>" + "Qui est l'auteur ? Retourne uniquement son nom de famille sous ce format : \\boxed{}."
user_msg_questionnaire = {"role" : "user", "content" : instructions}

In [13]:
# Extract the answer from openai raw output

def extract_answer(text):
    try:
        matches = re.findall(r'\\boxed{(.*?)}', text)
        if matches:
            return matches[-1]  
        else:
            return -1
    except Exception as e:
        return -1

In [14]:
# To maximize accuracy, let's ask the question many times and vote for the best answer

answer_list = []

for k in range(nb_repetitions):
    
    completion3 = openai.chat.completions.create(
        model=model,
        messages=[user_msg_questionnaire],
        temperature=temperature,
        max_tokens=max_tokens)

    raw_answer = completion3.choices[0].message.content

    parsed_answer = extract_answer(raw_answer)
    
    answer_list.append(parsed_answer)

In [15]:
# Hard vote for the best answer

def vote_best_answer(answer_list):
    
    counts = Counter(answer_list) 
    most_common = counts.most_common(2) 
    if len(most_common) == 1 or most_common[0][1] > most_common[1][1]:
        return most_common[0][0] 
    else:
        print('unable to extract a proper answer')
        return -1

In [16]:
answer = vote_best_answer(answer_list)
print(answer)

Camus
